In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import nltk
from nltk.corpus import stopwords
import re


In [2]:
df = pd.read_csv('/content/labeled_data.csv')
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:
#removing swahili stopwords from Byte Bullies Kenyan Dataset
sw_stopwords = pd.read_csv('stopwords_sw.csv') #Stopwords swahili

In [5]:
sw_stopwords.columns =['words']


In [6]:
sw_stopwords_list = sw_stopwords.words.tolist()

pat = r'\b(?:{})\b'.format('|'.join(sw_stopwords_list))

df['text'] = df['tweet'].str.replace(pat, '')

<ipython-input-6-3441c5d5e3a3>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['tweet'].str.replace(pat, '')


In [7]:
nltk.download('stopwords') #stopwords english
stop = set(stopwords.words('english'))

df["text"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0        !!! rt @mayasolovely: as woman complain cleani...
1        !!!!! rt @mleew17: boy dats cold...tyga dwn ba...
2        !!!!!!! rt @urkindofbrand dawg!!!! rt @80sbaby...
3        !!!!!!!!! rt @c_g_anderson: @viva_based look l...
4        !!!!!!!!!!!!! rt @shenikaroberts: the shit hea...
                               ...                        
24778    you's muthaf***in lie &#8220;@lifeasking: @20_...
24779     gone broke wrong heart baby, drove redneck crazy
24780    young buck wanna eat!!.. dat nigguh like i ain...
24781                    youu got wild bitches tellin lies
24782    ~~ruffled | ntac eileen dahlia - beautiful col...
Name: text, Length: 24783, dtype: object

In [9]:
def clean_text(text):
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emojis
                               u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
                               u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
                               u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Remove unnecessary content
    text = re.sub(r'http\S+', '', text)  # URLs
    text = re.sub(r'[^A-Za-z\s]+', '', text)  # Non-alphabetic characters except whitespaces

    return text

In [10]:
#Cleaning the text
df['cleaned_tweet'] = df['text'].apply(clean_text)
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,text,cleaned_tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,!!! RT @mayasolovely: As a woman you shouldn't...,RT mayasolovely As a woman you shouldnt compl...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,RT mleew boy dats coldtyga dwn bad for cuffin...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,RT UrKindOfBrand Dawg RT sbabylife You ever f...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,RT CGAnderson vivabased she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,RT ShenikaRoberts The shit you hear about me ...


In [11]:
#Assessing Imbalance
df['class'].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

In [ ]:
#label encoding
#labels = df.grouped_label.unique()

#label_dict = {}
#for index, label in enumerate(labels):
#    label_dict[label] = index

#label_dict

{'embarrassment_reputation_risk': 0,
 'insults_hate_speech': 1,
 'neutral_positive': 2,
 'sexualised_violence': 3}

In [ ]:
#Adding encoded labels
#df['group_index'] = df.grouped_label.replace(label_dict)

#df.head()

,text,label,grouped_label,group_index
0,kidero,trolling,embarrassment_reputation_risk,0
1,in stead of campaigning you were following rai...,insult,insults_hate_speech,1
2,stop lying tunajua kimoyomoyo you want to str...,insult,insults_hate_speech,1
3,goons,insult,insults_hate_speech,1
4,uhuru the most incompetent president in partne...,disinformation,embarrassment_reputation_risk,0


In [12]:
#Machine Learning
text = df["text"]
hate = df["class"]
X_train, X_test, y_train, y_test = train_test_split(text, hate, test_size=0.2)

# Create a count vectorizer to transform the text data into numerical feature vectors
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Create a Multinomial Naive Bayes classifier and train it on the data
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predict the class of the test data
predictions = clf.predict(X_test)

accuracy_score(y_test, predictions)

0.8601977002219084

In [13]:
# Random Forest Classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

# XGBoost Classifier
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)

# Predict the class of the test data
rfpredictions = rf_clf.predict(X_test)
xgbpredictions = xgb_clf.predict(X_test)

print("rf")
accuracy_score(y_test, rfpredictions)

print("xgb")
accuracy_score(y_test, xgbpredictions)

rf
xgb


0.9092192858583821

In [14]:
accuracy_score(y_test, rfpredictions)

0.8706879160782731

In [15]:
from sklearn.metrics import f1_score

# F1 score for XGBoost Classifier
nb_f1_score = f1_score(y_test, predictions, average='macro')
print("NaiveBayes F1 Score: ", nb_f1_score)

# F1 score for Random Forest Classifier
rf_f1_score = f1_score(y_test, rfpredictions, average='macro')
print("Random Forest F1 Score: ", rf_f1_score)

# F1 score for XGBoost Classifier
xgb_f1_score = f1_score(y_test, xgbpredictions, average='macro')
print("XGBoost F1 Score: ", xgb_f1_score)

NaiveBayes F1 Score:  0.5583937608559907
Random Forest F1 Score:  0.6201019291617819
XGBoost F1 Score:  0.7193890555823947
